In [1]:
import pandas as pd
import geopandas as gpd
import matsim

C:\Users\jdgoe\AppData\Local\Temp\ipykernel_11564\79990429.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


First, read the network and identify the links that are inside the area

In [16]:
network: gpd.GeoDataFrame = (matsim.read_network("../../../../output/berlin-v6.0-1pct/berlin-v6.0.output_network.xml.gz")
          .as_geo()
          )
area = gpd.read_file("../../../../input/shp/sbahn-ring.shp")[["NO", "geometry"]]
network

,length,freespeed,capacity,permlanes,oneway,modes,link_id,from_node,to_node,geometry
0,31.467991,4.166667,1200.0,1.0,1,"freight,ride,car",1,4864982246,262215600,"LINESTRING (801882.202 5830925.583, 801909.558..."
1,97.996533,6.944444,3200.0,2.0,1,"freight,ride,car",10,26703746,2951553394,"LINESTRING (796785.351 5828823.643, 796871.542..."
2,71.196902,2.777778,600.0,1.0,1,"freight,ride,car",100,27318315,587724650,"LINESTRING (781257.548 5812115.380, 781269.074..."
3,17.031152,4.166667,1200.0,1.0,1,"freight,ride,car",1000,29690627,29690626,"LINESTRING (793484.123 5832663.017, 793493.176..."
4,27.555616,6.944444,1800.0,1.5,1,"freight,ride,car",10000,1106607970,36265803,"LINESTRING (846334.221 5811290.686, 846310.802..."
...,...,...,...,...,...,...,...,...,...,...
202934,354.868966,12.236861,100000.0,1.0,1,pt,pt_9995,pt_500000065901,pt_500000087401,"LINESTRING (825628.437 5917191.910, 825971.759..."
202935,1353.018128,9.080659,100000.0,1.0,1,pt,pt_9996,pt_500000087401,pt_500000086501,"LINESTRING (825971.759 5917101.132, 827256.751..."
202936,1689.217448,8.082380,100000.0,1.0,1,pt,pt_9997,pt_500000086501,pt_500000086601,"LINESTRING (827256.751 5916674.443, 828269.225..."
202937,1600.265408,7.656772,100000.0,1.0,1,pt,pt_9998,pt_500000086601,pt_500000083701,"LINESTRING (828269.225 5915320.761, 828775.804..."


In [35]:
network_intersected = network.sjoin(
    area, how="left", predicate="within"
).assign(in_area = lambda row: row["NO"] == 0)[["link_id", "in_area"]]
all_links_in_area = list(network_intersected.query("in_area == True")["link_id"])

In [39]:
# This runs for an hour!
events = matsim.event_reader("../../../../output/berlin-v6.0-1pct/berlin-v6.0.output_events.xml.gz")
vehicles_tolled = []
for event in events:
    if event["type"] == "entered link" and event["link"] in all_links_in_area:
        vehicles_tolled.append(event["vehicle"])
unique_vehicles_tolled = set(vehicles_tolled)
print(len(unique_vehicles_tolled))


29215


Result: 29215, including pt!

Now find out of which types those vehicles are

In [53]:
all_vehicles = matsim.vehicle_reader("../../../../output/berlin-v6.0-1pct/berlin-v6.0.output_vehicles.xml.gz").vehicles

df_tolled_vehicles = (pd.DataFrame(data={"id": list(unique_vehicles_tolled)})
                      .merge(all_vehicles, on="id", how="left")
                      )
df_tolled_vehicles = df_tolled_vehicles[~df_tolled_vehicles["id"].str.startswith("pt")]
df_tolled_vehicles["type"].value_counts()


type
car        6395
freight     888
Name: count, dtype: int64

car: 6395\
freight: 888\
total: 7283

In Ihab's Berlin scenario, the agents paid a total toll of ca. 2M EUR, upscaled to 100%\
When we say we have approx. 7300 cars tolled, the resulting price should be $2000000 EUR/7300*100=2,74 EUR$